## Alica

In [ ]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from fastai.text import *
import html

In [ ]:
BOS = 'xbos'  # beginning-of-sentence tag
FLD = 'xfld'  # data field tag

PATH=Path('data/alica/')

## Standardize format

In [ ]:
CLAS_PATH=Path('data/alica_clas/')
CLAS_PATH.mkdir(exist_ok=True)

LM_PATH=Path('data/alica_lm/')
LM_PATH.mkdir(exist_ok=True)

In [ ]:
CLASSES = ['good', 'neutral', 'bad']
d = {"good":2,"neutral":1,"bad":0}
#d = {"good":1,"neutral":0}
#d = {"good":1,"bad":0}
col_names = ['labels','text']

In [ ]:
(CLAS_PATH/'classes.txt').open('w').writelines(f'{o}\n' for o in CLASSES)

In [ ]:
df = pd.read_csv(PATH/"public.tsv", sep = "\t", header = None, quoting=csv.QUOTE_NONE)
df.fillna("", inplace=True)
df["text"] = df[1]+"|"+df[2]+"|"+df[3]+"|"+df[5]
df["id"] = df[0]
df["num"] = df[4]
df["labels"] = [0]*len(df)

df.to_csv(CLAS_PATH/"test.csv", columns=col_names+["id","num"], sep="\t", header=False, index=False)

In [ ]:
df = pd.read_csv(PATH/"final.tsv", sep = "\t", header = None, quoting=csv.QUOTE_NONE)
df.fillna("", inplace=True)
df["text"] = df[1]+"|"+df[2]+"|"+df[3]+"|"+df[5]
df["id"] = df[0]
df["num"] = df[4]
df["labels"] = [0]*len(df)

df.to_csv(CLAS_PATH/"test_.csv", columns=col_names+["id","num"], sep="\t", header=False, index=False)

In [ ]:
#folds neutral=bad
df = pd.read_csv(PATH/"train.tsv", sep = "\t", header = None, quoting=csv.QUOTE_NONE)
df.head()

df.fillna("", inplace=True)
df["text"] = df[1]+"|"+df[2]+"|"+df[3]+"|"+df[5]
df["id"] = df[0]
df["num"] = df[4]
df["labels"] = df[6].apply(lambda x: d[x.replace("neutral","bad")])

from sklearn.model_selection import KFold
kf = KFold(n_splits=10, random_state=42, shuffle=True)
i = 0
for train_index, valid_index in kf.split(df):
    dff, dfv = df.iloc[train_index,:], df.iloc[valid_index,:]
    
    dff.to_csv(CLAS_PATH/f"train_{i}.csv", columns=col_names+["id","num"], sep="\t", header=False, index=False)
    dfv.to_csv(CLAS_PATH/f"valid_{i}.csv", columns=col_names+["id","num"], sep="\t", header=False, index=False)
    
    i+=1


In [ ]:
#folds neutral=good
df = pd.read_csv(PATH/"train.tsv", sep = "\t", header = None, quoting=csv.QUOTE_NONE)
df.head()

df.fillna("", inplace=True)
df["text"] = df[1]+"|"+df[2]+"|"+df[3]+"|"+df[5]
df["id"] = df[0]
df["num"] = df[4]
df["labels"] = df[6].apply(lambda x: d[x.replace("neutral","good")])

from sklearn.model_selection import KFold
kf = KFold(n_splits=10, random_state=42, shuffle=True)
i = 0
for train_index, valid_index in kf.split(df):
    dff, dfv = df.iloc[train_index,:], df.iloc[valid_index,:]
    
    dff.to_csv(CLAS_PATH/f"train_{i}_.csv", columns=col_names+["id","num"], sep="\t", header=False, index=False)
    dfv.to_csv(CLAS_PATH/f"valid_{i}_.csv", columns=col_names+["id","num"], sep="\t", header=False, index=False)
    
    i+=1

In [ ]:
#folds 3
df = pd.read_csv(PATH/"train.tsv", sep = "\t", header = None, quoting=csv.QUOTE_NONE)
df.head()

df.fillna("", inplace=True)
df["text"] = df[1]+"|"+df[2]+"|"+df[3]+"|"+df[5]
df["id"] = df[0]
df["num"] = df[4]
df["labels"] = df[6].apply(lambda x: d[x])

from sklearn.model_selection import KFold
kf = KFold(n_splits=10, random_state=42, shuffle=True)
i = 0
for train_index, valid_index in kf.split(df):
    dff, dfv = df.iloc[train_index,:], df.iloc[valid_index,:]
    
    dff.to_csv(CLAS_PATH/f"train_{i}_3.csv", columns=col_names+["id","num"], sep="\t", header=False, index=False)
    dfv.to_csv(CLAS_PATH/f"valid_{i}_3.csv", columns=col_names+["id","num"], sep="\t", header=False, index=False)
    
    i+=1

In [ ]:
#folds no neutral
df = pd.read_csv(PATH/"train.tsv", sep = "\t", header = None, quoting=csv.QUOTE_NONE)

df.fillna("", inplace=True)
df = df[df[6]!="neutral"]

df["text"] = df[1]+"|"+df[2]+"|"+df[3]+"|"+df[5]
df["id"] = df[0]
df["num"] = df[4]
df["labels"] = df[6].apply(lambda x: d[x])

from sklearn.model_selection import KFold
kf = KFold(n_splits=10, random_state=42, shuffle=True)
i = 0
for train_index, valid_index in kf.split(df):
    dff, dfv = df.iloc[train_index,:], df.iloc[valid_index,:]
    
    dff.to_csv(CLAS_PATH/f"train_{i}_n.csv", columns=col_names+["id","num"], sep="\t", header=False, index=False)
    dfv.to_csv(CLAS_PATH/f"valid_{i}_n.csv", columns=col_names+["id","num"], sep="\t", header=False, index=False)
    
    i+=1

In [ ]:
#folds reg
df = pd.read_csv(PATH/"train.tsv", sep = "\t", header = None, quoting=csv.QUOTE_NONE)

df.fillna("", inplace=True)
df = df[df[6]!="neutral"]

df["text"] = df[1]+"|"+df[2]+"|"+df[3]+"|"+df[5]
df["id"] = df[0]
df["num"] = df[4]

res = []
for x,y in zip(df[6], df[7]):
    if x == "good":
        t = 1+y
    if x == "neutral":
        t = 1 
    if x == "bad":
        t = 1-y
    res.append(t)

df["labels"] = res

from sklearn.model_selection import KFold
kf = KFold(n_splits=10, random_state=42, shuffle=True)
i = 0
for train_index, valid_index in kf.split(df):
    dff, dfv = df.iloc[train_index,:], df.iloc[valid_index,:]
    
    dff.to_csv(CLAS_PATH/f"train_{i}_r.csv", columns=col_names+["id","num"], sep="\t", header=False, index=False)
    dfv.to_csv(CLAS_PATH/f"valid_{i}_r.csv", columns=col_names+["id","num"], sep="\t", header=False, index=False)
    
    i+=1

## OpenSubtitles

In [ ]:
df = pd.read_csv(PATH/"OpenSubtitles2016.en-ru.ru", sep = "\t", header = None, quoting=csv.QUOTE_NONE)

In [ ]:
texts = df[0]
len(texts)

In [ ]:
trn_texts,val_texts = sklearn.model_selection.train_test_split(texts, test_size=0.1)

In [ ]:
len(trn_texts), len(val_texts)

In [ ]:
df_trn = pd.DataFrame({'text':trn_texts, 'labels':[0]*len(trn_texts)}, columns=col_names)
df_val = pd.DataFrame({'text':val_texts, 'labels':[0]*len(val_texts)}, columns=col_names)

df_trn.to_csv(LM_PATH/'train.csv', header=False, index=False)
df_val.to_csv(LM_PATH/'test.csv', header=False, index=False)

## Language model tokens

In [ ]:
chunksize=96000

In [ ]:
re1 = re.compile(r'  +')

def fixup(x):
    x = x.replace(u'\xa0', u' ')
    x = x.replace('|', '\n')
    x = x.replace('\n\n\n', '\n')
    x = x.replace('\n\n', '\n')
    x = x.replace(". . .","...")
    x = " , ".join(x.split(","))
    x = " . ".join(x.split("."))
    x = x.replace(".  .  .","...")
    
    return re1.sub(' ', html.unescape(x))

In [ ]:
def get_texts(df):
    #labels = df[0].values.astype(np.int64)
    labels = df[0].values.astype(float) #for reg
    texts = '\n' + df[1].astype(str)
    texts = texts.apply(fixup).values.astype(str)

    tok = Tokenizer().proc_all_mp(partition_by_cores(texts), 'xx')
    return tok, list(labels)

In [ ]:
def get_all(df):
    tok, labels = [], []
    for i, r in tqdm(enumerate(df)):
        tok_, labels_ = get_texts(r)
        tok += tok_;
        labels += labels_
    return tok, labels

In [ ]:
df_val = pd.read_csv(LM_PATH/'test.csv', header=None, chunksize=chunksize)
tok_val, val_labels = get_all(df_val)

In [ ]:
df_trn = pd.read_csv(LM_PATH/'train.csv', header=None, chunksize=chunksize)
tok_trn, trn_labels = get_all(df_trn)

In [ ]:
from random import choice
choice(tok_trn)

In [ ]:
(LM_PATH/'tmp').mkdir(exist_ok=True)

In [ ]:
np.save(LM_PATH/'tmp'/'tok_trn_0.npy', tok_trn[:4000000])
np.save(LM_PATH/'tmp'/'tok_trn_1.npy', tok_trn[4000000:8000000])
np.save(LM_PATH/'tmp'/'tok_trn_2.npy', tok_trn[8000000:10000000])
np.save(LM_PATH/'tmp'/'tok_trn_3.npy', tok_trn[10000000:12000000])
np.save(LM_PATH/'tmp'/'tok_trn_4.npy', tok_trn[12000000:14000000])
np.save(LM_PATH/'tmp'/'tok_trn_5.npy', tok_trn[14000000:15000000])
np.save(LM_PATH/'tmp'/'tok_trn_6.npy', tok_trn[15000000:])
np.save(LM_PATH/'tmp'/'tok_val.npy', tok_val)

In [ ]:
tok_val = np.load(LM_PATH/'tmp'/'tok_val.npy')

In [ ]:
tok_trn = np.array([])
for i in tqdm(range(7)):
    tok_trn = np.append(tok_trn, np.load(LM_PATH/'tmp'/f'tok_trn_{i}.npy'))

In [ ]:
freq = Counter(p for o in tok_trn for p in o)
freq.most_common(25)

In [ ]:
max_vocab = 60000
min_freq = 2

In [ ]:
itos = [o for o,c in freq.most_common(max_vocab) if c>min_freq]
itos.insert(0, '_pad_')
itos.insert(0, '_unk_')

In [ ]:
stoi = collections.defaultdict(lambda:0, {v:k for k,v in enumerate(itos)})
len(itos)

In [ ]:
trn_lm = np.array([[stoi[o] for o in p] for p in tok_trn])
val_lm = np.array([[stoi[o] for o in p] for p in tok_val])

In [ ]:
np.save(LM_PATH/'tmp'/'trn_ids.npy', trn_lm)
np.save(LM_PATH/'tmp'/'val_ids.npy', val_lm)
pickle.dump(itos, open(LM_PATH/'tmp'/'itos.pkl', 'wb'))

In [ ]:
trn_lm = np.load(LM_PATH/'tmp'/'trn_ids.npy')
val_lm = np.load(LM_PATH/'tmp'/'val_ids.npy')
itos = pickle.load(open(LM_PATH/'tmp'/'itos.pkl', 'rb'))

In [ ]:
vs=len(itos)
vs,len(trn_lm)

## Language model

In [ ]:
em_sz,nh,nl = 400,1150,3

wd=1e-7
bptt=70
bs=52
opt_fn = partial(optim.Adam, betas=(0.8, 0.99))

In [ ]:
trn_dl = LanguageModelLoader(np.concatenate(trn_lm), bs, bptt)
val_dl = LanguageModelLoader(np.concatenate(val_lm), bs, bptt)
md = LanguageModelData(PATH, 1, vs, trn_dl, val_dl, bs=bs, bptt=bptt)

In [ ]:
drops = np.array([0.25, 0.1, 0.2, 0.02, 0.15])*0.7

In [ ]:
learner= md.get_model(opt_fn, em_sz, nh, nl, 
    dropouti=drops[0], dropout=drops[1], wdrop=drops[2], dropoute=drops[3], dropouth=drops[4])

learner.metrics = [accuracy]
learner.unfreeze()

In [ ]:
lr=1e-3
lrs = lr

In [ ]:
learner.fit(lrs/2, 1, wds=wd, use_clr=(32,2), cycle_len=1)

In [ ]:
learner.save('lm_last_ft')

In [ ]:
learner.load('lm_last_ft')

In [ ]:
learner.unfreeze()

In [ ]:
learner.fit(lrs/2, 1, wds=wd, use_clr=(32,2), cycle_len=1)

In [ ]:
learner.load('lm0')

In [ ]:
learner.fit(lrs, 1, wds=wd, use_clr=(20,10), cycle_len=5)

In [ ]:
#3.980186   3.859496   0.364186
learner.save('lm1')

In [ ]:
learner.save_encoder('lm_enc1')

In [ ]:
learner.load('lm1')

In [ ]:
learner.fit(lrs, 1, wds=wd, use_clr=(20,10), cycle_len=5)

In [ ]:
learner.sched.plot_loss()

In [ ]:
learner.save('lm2')
learner.save_encoder('lm_enc2')
#3.961754   3.840744   0.365777

## Classifier tokens

In [ ]:
#df_trn = pd.read_csv(CLAS_PATH/'train.csv', header=None, chunksize=chunksize, sep="\t", quoting=csv.QUOTE_NONE)
#df_val = pd.read_csv(CLAS_PATH/'valid.csv', header=None, chunksize=chunksize, sep="\t", quoting=csv.QUOTE_NONE)
#df_tst = pd.read_csv(CLAS_PATH/'test.csv', header=None, chunksize=chunksize, sep="\t", quoting=csv.QUOTE_NONE)
df_tst = pd.read_csv(CLAS_PATH/'test_.csv', header=None, chunksize=chunksize, sep="\t", quoting=csv.QUOTE_NONE)

In [ ]:
#from random import choice
#choice(tok_tst)

In [ ]:
#tok_trn, trn_labels = get_all(df_trn)
#tok_val, val_labels = get_all(df_val)
tok_tst, tst_labels = get_all(df_tst)


In [ ]:
(CLAS_PATH/'tmp').mkdir(exist_ok=True)

#np.save(CLAS_PATH/'tmp'/'tok_trn.npy', tok_trn)
#np.save(CLAS_PATH/'tmp'/'tok_val.npy', tok_val)
#np.save(CLAS_PATH/'tmp'/'tok_tst.npy', tok_tst)
np.save(CLAS_PATH/'tmp'/'tok_tst_.npy', tok_tst)

#np.save(CLAS_PATH/'tmp'/'trn_labels.npy', trn_labels)
#np.save(CLAS_PATH/'tmp'/'val_labels.npy', val_labels)
#np.save(CLAS_PATH/'tmp'/'tst_labels.npy', tst_labels)
np.save(CLAS_PATH/'tmp'/'tst_labels_.npy', tst_labels)

In [ ]:
#tok_trn = np.load(CLAS_PATH/'tmp'/'tok_trn.npy')
#tok_val = np.load(CLAS_PATH/'tmp'/'tok_val.npy')
#tok_tst = np.load(CLAS_PATH/'tmp'/'tok_tst.npy')
tok_tst = np.load(CLAS_PATH/'tmp'/'tok_tst_.npy')

In [ ]:
itos = pickle.load((LM_PATH/'tmp'/'itos.pkl').open('rb'))
stoi = collections.defaultdict(lambda:0, {v:k for k,v in enumerate(itos)})
len(itos)

In [ ]:
#trn_clas = np.array([[stoi[o] for o in p] for p in tok_trn])
#val_clas = np.array([[stoi[o] for o in p] for p in tok_val])
tst_clas = np.array([[stoi[o] for o in p] for p in tok_tst])

In [ ]:
#np.save(CLAS_PATH/'tmp'/'trn_ids.npy', trn_clas)
#np.save(CLAS_PATH/'tmp'/'val_ids.npy', val_clas)
#np.save(CLAS_PATH/'tmp'/'tst_ids.npy', tst_clas)
np.save(CLAS_PATH/'tmp'/'tst_ids_.npy', tst_clas)

In [ ]:
#folds nuetral=bad
chunksize=96000

itos = pickle.load((LM_PATH/'tmp'/'itos.pkl').open('rb'))
stoi = collections.defaultdict(lambda:0, {v:k for k,v in enumerate(itos)})

for i in tqdm(range(10)):
    df_trn = pd.read_csv(CLAS_PATH/f'train_{i}.csv', header=None, chunksize=chunksize, sep="\t", quoting=csv.QUOTE_NONE)
    df_val = pd.read_csv(CLAS_PATH/f'valid_{i}.csv', header=None, chunksize=chunksize, sep="\t", quoting=csv.QUOTE_NONE)
    
    tok_trn, trn_labels = get_all(df_trn)
    tok_val, val_labels = get_all(df_val)
    
    np.save(CLAS_PATH/'tmp'/f'tok_trn_{i}.npy', tok_trn)
    np.save(CLAS_PATH/'tmp'/f'tok_val_{i}.npy', tok_val)

    np.save(CLAS_PATH/'tmp'/f'trn_labels_{i}.npy', trn_labels)
    np.save(CLAS_PATH/'tmp'/f'val_labels_{i}.npy', val_labels)
    
    trn_clas = np.array([[stoi[o] for o in p] for p in tok_trn])
    val_clas = np.array([[stoi[o] for o in p] for p in tok_val])

    np.save(CLAS_PATH/'tmp'/f'trn_ids_{i}.npy', trn_clas)
    np.save(CLAS_PATH/'tmp'/f'val_ids_{i}.npy', val_clas)

In [ ]:
#folds nuetral=good
chunksize=96000

itos = pickle.load((LM_PATH/'tmp'/'itos.pkl').open('rb'))
stoi = collections.defaultdict(lambda:0, {v:k for k,v in enumerate(itos)})

for i in tqdm(range(10)):
    df_trn = pd.read_csv(CLAS_PATH/f'train_{i}_.csv', header=None, chunksize=chunksize, sep="\t", quoting=csv.QUOTE_NONE)
    df_val = pd.read_csv(CLAS_PATH/f'valid_{i}_.csv', header=None, chunksize=chunksize, sep="\t", quoting=csv.QUOTE_NONE)
    
    tok_trn, trn_labels = get_all(df_trn)
    tok_val, val_labels = get_all(df_val)
    
    np.save(CLAS_PATH/'tmp'/f'tok_trn_{i}_.npy', tok_trn)
    np.save(CLAS_PATH/'tmp'/f'tok_val_{i}_.npy', tok_val)

    np.save(CLAS_PATH/'tmp'/f'trn_labels_{i}_.npy', trn_labels)
    np.save(CLAS_PATH/'tmp'/f'val_labels_{i}_.npy', val_labels)
    
    trn_clas = np.array([[stoi[o] for o in p] for p in tok_trn])
    val_clas = np.array([[stoi[o] for o in p] for p in tok_val])

    np.save(CLAS_PATH/'tmp'/f'trn_ids_{i}_.npy', trn_clas)
    np.save(CLAS_PATH/'tmp'/f'val_ids_{i}_.npy', val_clas)

In [ ]:
#folds 3
chunksize=96000

itos = pickle.load((LM_PATH/'tmp'/'itos.pkl').open('rb'))
stoi = collections.defaultdict(lambda:0, {v:k for k,v in enumerate(itos)})

for i in tqdm(range(10)):
    df_trn = pd.read_csv(CLAS_PATH/f'train_{i}_3.csv', header=None, chunksize=chunksize, sep="\t", quoting=csv.QUOTE_NONE)
    df_val = pd.read_csv(CLAS_PATH/f'valid_{i}_3.csv', header=None, chunksize=chunksize, sep="\t", quoting=csv.QUOTE_NONE)
    
    tok_trn, trn_labels = get_all(df_trn)
    tok_val, val_labels = get_all(df_val)
    
    np.save(CLAS_PATH/'tmp'/f'tok_trn_{i}_3.npy', tok_trn)
    np.save(CLAS_PATH/'tmp'/f'tok_val_{i}_3.npy', tok_val)

    np.save(CLAS_PATH/'tmp'/f'trn_labels_{i}_3.npy', trn_labels)
    np.save(CLAS_PATH/'tmp'/f'val_labels_{i}_3.npy', val_labels)
    
    trn_clas = np.array([[stoi[o] for o in p] for p in tok_trn])
    val_clas = np.array([[stoi[o] for o in p] for p in tok_val])

    np.save(CLAS_PATH/'tmp'/f'trn_ids_{i}_3.npy', trn_clas)
    np.save(CLAS_PATH/'tmp'/f'val_ids_{i}_3.npy', val_clas)

In [ ]:
#folds no neutral
chunksize=96000

itos = pickle.load((LM_PATH/'tmp'/'itos.pkl').open('rb'))
stoi = collections.defaultdict(lambda:0, {v:k for k,v in enumerate(itos)})

for i in tqdm(range(10)):
    df_trn = pd.read_csv(CLAS_PATH/f'train_{i}_n.csv', header=None, chunksize=chunksize, sep="\t", quoting=csv.QUOTE_NONE)
    df_val = pd.read_csv(CLAS_PATH/f'valid_{i}_n.csv', header=None, chunksize=chunksize, sep="\t", quoting=csv.QUOTE_NONE)
    
    tok_trn, trn_labels = get_all(df_trn)
    tok_val, val_labels = get_all(df_val)
    
    np.save(CLAS_PATH/'tmp'/f'tok_trn_{i}_n.npy', tok_trn)
    np.save(CLAS_PATH/'tmp'/f'tok_val_{i}_n.npy', tok_val)

    np.save(CLAS_PATH/'tmp'/f'trn_labels_{i}_n.npy', trn_labels)
    np.save(CLAS_PATH/'tmp'/f'val_labels_{i}_n.npy', val_labels)
    
    trn_clas = np.array([[stoi[o] for o in p] for p in tok_trn])
    val_clas = np.array([[stoi[o] for o in p] for p in tok_val])

    np.save(CLAS_PATH/'tmp'/f'trn_ids_{i}_n.npy', trn_clas)
    np.save(CLAS_PATH/'tmp'/f'val_ids_{i}_n.npy', val_clas)

In [ ]:
#folds reg
chunksize=96000

itos = pickle.load((LM_PATH/'tmp'/'itos.pkl').open('rb'))
stoi = collections.defaultdict(lambda:0, {v:k for k,v in enumerate(itos)})

for i in tqdm(range(10)):
    df_trn = pd.read_csv(CLAS_PATH/f'train_{i}_r.csv', header=None, chunksize=chunksize, sep="\t", quoting=csv.QUOTE_NONE)
    df_val = pd.read_csv(CLAS_PATH/f'valid_{i}_r.csv', header=None, chunksize=chunksize, sep="\t", quoting=csv.QUOTE_NONE)
    
    tok_trn, trn_labels = get_all(df_trn)
    tok_val, val_labels = get_all(df_val)
    
    np.save(CLAS_PATH/'tmp'/f'tok_trn_{i}_r.npy', tok_trn)
    np.save(CLAS_PATH/'tmp'/f'tok_val_{i}_r.npy', tok_val)

    np.save(CLAS_PATH/'tmp'/f'trn_labels_{i}_r.npy', trn_labels)
    np.save(CLAS_PATH/'tmp'/f'val_labels_{i}_r.npy', val_labels)
    
    trn_clas = np.array([[stoi[o] for o in p] for p in tok_trn])
    val_clas = np.array([[stoi[o] for o in p] for p in tok_val])

    np.save(CLAS_PATH/'tmp'/f'trn_ids_{i}_r.npy', trn_clas)
    np.save(CLAS_PATH/'tmp'/f'val_ids_{i}_r.npy', val_clas)

## Predict

In [ ]:
path = "/mnt/6676114C76111E7D/Kaggle/Alica/"

In [ ]:
#folds neutral=bad
itos = pickle.load((LM_PATH/'tmp'/'itos.pkl').open('rb'))
stoi = collections.defaultdict(lambda:0, {v:k for k,v in enumerate(itos)})

bptt,em_sz,nh,nl = 70,400,1150,3
vs = len(itos)
opt_fn = partial(optim.Adam, betas=(0.8, 0.99))
bs = 48
c = 2
dps = np.array([0.4,0.5,0.05,0.3,0.4])*0.75

lr=1e-3
lrm = 2.6
lrs = np.array([lr/(lrm**4), lr/(lrm**3), lr/(lrm**2), lr/lrm, lr])
#lrs=np.array([1e-4,1e-4,1e-4,1e-3,1e-2])

tst_clas = np.load(CLAS_PATH/'tmp'/'tst_ids_.npy')
tst_labels = np.squeeze(np.load(CLAS_PATH/'tmp'/'tst_labels_.npy'))
tst_ds = TextDataset(tst_clas, tst_labels)
tst_samp = SimpleSampler(tst_clas)
tst_dl = DataLoader(tst_ds, bs, transpose=True, num_workers=1, pad_idx=1, sampler=tst_samp)

for i in tqdm(range(10)):
    trn_clas = np.load(CLAS_PATH/'tmp'/f'trn_ids_{i}.npy')
    val_clas = np.load(CLAS_PATH/'tmp'/f'val_ids_{i}.npy')
    
    trn_labels = np.squeeze(np.load(CLAS_PATH/'tmp'/f'trn_labels_{i}.npy'))
    val_labels = np.squeeze(np.load(CLAS_PATH/'tmp'/f'val_labels_{i}.npy'))

    trn_ds = TextDataset(trn_clas, trn_labels)
    val_ds = TextDataset(val_clas, val_labels)
    
    trn_samp = SortishSampler(trn_clas, key=lambda x: len(trn_clas[x]), bs=bs//2)
    val_samp = SortSampler(val_clas, key=lambda x: len(val_clas[x]))
    
    trn_dl = DataLoader(trn_ds, bs//2, transpose=True, num_workers=1, pad_idx=1, sampler=trn_samp)
    val_dl = DataLoader(val_ds, bs, transpose=True, num_workers=1, pad_idx=1, sampler=val_samp)
    
    md = ModelData(PATH, trn_dl, val_dl, tst_dl)

    m = get_rnn_classifer(bptt, 20*70, c, vs, emb_sz=em_sz, n_hid=nh, n_layers=nl, pad_token=1,
              layers=[em_sz*3, 50, c], drops=[dps[4], 0.1],
              dropouti=dps[0], wdrop=dps[1], dropoute=dps[2], dropouth=dps[3])

    opt_fn = partial(optim.Adam, betas=(0.7, 0.99))

    learn = RNN_Learner(md, TextModel(to_gpu(m)), opt_fn=opt_fn)
    learn.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
    learn.clip=25.
    learn.metrics = [accuracy]

#     wd = 0
#     learn.load_encoder('lm_enc1')
#     learn.freeze_to(-1)
#     learn.fit(lrs, 1, wds=wd, cycle_len=1, use_clr=(8,3))
#     learn.save(f'fold_{i}')

#     learn.freeze_to(-2)
#     learn.fit(lrs, 1, wds=wd, cycle_len=1, use_clr=(8,3))
#     learn.save(f'fold_{i}')
    
#     wd = 1e-7
#     learn.unfreeze()
#     learn.fit(lrs, 1, wds=wd, cycle_len=1, use_clr=(8,3))
#     learn.save(f'fold_{i}')
    
#     learn.fit(lrs, 2, wds=wd, cycle_len=1)
#     learn.save(f'fold_{i}')
    learn.load(f'./10_folds_fwd_bad_86283/fold_{i}')
    
    wd = 1e-7
    learn.unfreeze()
    learn.fit(lrs, 1, wds=wd, cycle_len=1, use_clr=(10,10,0.95,0.85))
    learn.save(path+f'fold_{i}')

    preds = learn.predict(is_test=True)
    
    df = pd.read_csv(CLAS_PATH/'test_.csv', header=None, sep = "\t", quoting=csv.QUOTE_NONE)
    df["prob"] = [c[1] for c in np.exp(preds)] 
    df.sort_values(by=[2,"prob"], inplace=True, ascending=[True,False])
    df.to_csv(f"./blend1/_fold_{i}.tsv",columns=[2,3,"prob"],index=False,sep="\t",header=False)

In [ ]:
#folds neutral=good
itos = pickle.load((LM_PATH/'tmp'/'itos.pkl').open('rb'))
stoi = collections.defaultdict(lambda:0, {v:k for k,v in enumerate(itos)})

bptt,em_sz,nh,nl = 70,400,1150,3
vs = len(itos)
opt_fn = partial(optim.Adam, betas=(0.8, 0.99))
bs = 48
c = 2
dps = np.array([0.4,0.5,0.05,0.3,0.4])*0.75

lr=1e-3
lrm = 2.6
lrs = np.array([lr/(lrm**4), lr/(lrm**3), lr/(lrm**2), lr/lrm, lr])
#lrs=np.array([1e-4,1e-4,1e-4,1e-3,1e-2])

tst_clas = np.load(CLAS_PATH/'tmp'/'tst_ids_.npy')
tst_labels = np.squeeze(np.load(CLAS_PATH/'tmp'/'tst_labels_.npy'))
tst_ds = TextDataset(tst_clas, tst_labels)
tst_samp = SimpleSampler(tst_clas)
tst_dl = DataLoader(tst_ds, bs, transpose=True, num_workers=1, pad_idx=1, sampler=tst_samp)

for i in tqdm(range(10)):
    trn_clas = np.load(CLAS_PATH/'tmp'/f'trn_ids_{i}_.npy')
    val_clas = np.load(CLAS_PATH/'tmp'/f'val_ids_{i}_.npy')
    
    trn_labels = np.squeeze(np.load(CLAS_PATH/'tmp'/f'trn_labels_{i}_.npy'))
    val_labels = np.squeeze(np.load(CLAS_PATH/'tmp'/f'val_labels_{i}_.npy'))

    trn_ds = TextDataset(trn_clas, trn_labels)
    val_ds = TextDataset(val_clas, val_labels)
    
    trn_samp = SortishSampler(trn_clas, key=lambda x: len(trn_clas[x]), bs=bs//2)
    val_samp = SortSampler(val_clas, key=lambda x: len(val_clas[x]))
    
    trn_dl = DataLoader(trn_ds, bs//2, transpose=True, num_workers=1, pad_idx=1, sampler=trn_samp)
    val_dl = DataLoader(val_ds, bs, transpose=True, num_workers=1, pad_idx=1, sampler=val_samp)
    
    md = ModelData(PATH, trn_dl, val_dl, tst_dl)

    m = get_rnn_classifer(bptt, 20*70, c, vs, emb_sz=em_sz, n_hid=nh, n_layers=nl, pad_token=1,
              layers=[em_sz*3, 50, c], drops=[dps[4], 0.1],
              dropouti=dps[0], wdrop=dps[1], dropoute=dps[2], dropouth=dps[3])

    opt_fn = partial(optim.Adam, betas=(0.7, 0.99))

    learn = RNN_Learner(md, TextModel(to_gpu(m)), opt_fn=opt_fn)
    learn.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
    learn.clip=25.
    learn.metrics = [accuracy]

#     wd = 0
#     learn.load_encoder('lm_enc1')
#     learn.freeze_to(-1)
#     learn.fit(lrs, 1, wds=wd, cycle_len=1, use_clr=(8,3))
#     learn.save(f'fold_{i}_')

#     learn.freeze_to(-2)
#     learn.fit(lrs, 1, wds=wd, cycle_len=1, use_clr=(8,3))
#     learn.save(f'fold_{i}_')
    
#     wd = 1e-7
#     learn.unfreeze()
#     learn.fit(lrs, 1, wds=wd, cycle_len=1, use_clr=(8,3))
#     learn.save(f'fold_{i}_')
    
#     learn.fit(lrs, 1, wds=wd, cycle_len=1)
#     learn.save(f'fold_{i}_')
    learn.load(f'./10_folds_fwd_good_85739/fold_{i}_')
    
    wd = 1e-7
    learn.unfreeze()
    learn.fit(lrs, 1, wds=wd, cycle_len=1, use_clr=(10,10,0.95,0.85))
    learn.save(path+f'fold_{i}_')
        
    preds = learn.predict(is_test=True)
    
    df = pd.read_csv(CLAS_PATH/'test_.csv', header=None, sep = "\t", quoting=csv.QUOTE_NONE)
    df["prob"] = [c[1] for c in np.exp(preds)] 
    df.sort_values(by=[2,"prob"], inplace=True, ascending=[True,False])
    df.to_csv(f"./blend1/_fold_{i}_.tsv",columns=[2,3,"prob"],index=False,sep="\t",header=False)

In [ ]:
#folds 3
itos = pickle.load((LM_PATH/'tmp'/'itos.pkl').open('rb'))
stoi = collections.defaultdict(lambda:0, {v:k for k,v in enumerate(itos)})

bptt,em_sz,nh,nl = 70,400,1150,3
vs = len(itos)
opt_fn = partial(optim.Adam, betas=(0.8, 0.99))
bs = 48
c = 3
dps = np.array([0.4,0.5,0.05,0.3,0.4])*0.75

lr=1e-3
lrm = 2.6
lrs = np.array([lr/(lrm**4), lr/(lrm**3), lr/(lrm**2), lr/lrm, lr])
#lrs=np.array([1e-4,1e-4,1e-4,1e-3,1e-2])

tst_clas = np.load(CLAS_PATH/'tmp'/'tst_ids_.npy')
tst_labels = np.squeeze(np.load(CLAS_PATH/'tmp'/'tst_labels_.npy'))
tst_ds = TextDataset(tst_clas, tst_labels)
tst_samp = SimpleSampler(tst_clas)
tst_dl = DataLoader(tst_ds, bs, transpose=True, num_workers=1, pad_idx=1, sampler=tst_samp)

for i in tqdm(range(10)):
    trn_clas = np.load(CLAS_PATH/'tmp'/f'trn_ids_{i}_3.npy')
    val_clas = np.load(CLAS_PATH/'tmp'/f'val_ids_{i}_3.npy')
    
    trn_labels = np.squeeze(np.load(CLAS_PATH/'tmp'/f'trn_labels_{i}_3.npy'))
    val_labels = np.squeeze(np.load(CLAS_PATH/'tmp'/f'val_labels_{i}_3.npy'))

    trn_ds = TextDataset(trn_clas, trn_labels)
    val_ds = TextDataset(val_clas, val_labels)
    
    trn_samp = SortishSampler(trn_clas, key=lambda x: len(trn_clas[x]), bs=bs//2)
    val_samp = SortSampler(val_clas, key=lambda x: len(val_clas[x]))
    
    trn_dl = DataLoader(trn_ds, bs//2, transpose=True, num_workers=1, pad_idx=1, sampler=trn_samp)
    val_dl = DataLoader(val_ds, bs, transpose=True, num_workers=1, pad_idx=1, sampler=val_samp)
    
    md = ModelData(PATH, trn_dl, val_dl, tst_dl)

    m = get_rnn_classifer(bptt, 20*70, c, vs, emb_sz=em_sz, n_hid=nh, n_layers=nl, pad_token=1,
              layers=[em_sz*3, 50, c], drops=[dps[4], 0.1],
              dropouti=dps[0], wdrop=dps[1], dropoute=dps[2], dropouth=dps[3])

    opt_fn = partial(optim.Adam, betas=(0.7, 0.99))

    learn = RNN_Learner(md, TextModel(to_gpu(m)), opt_fn=opt_fn)
    learn.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
    learn.clip=25.
    learn.metrics = [accuracy]

#     wd = 0
#     learn.load_encoder('lm_enc1')
#     learn.freeze_to(-1)
#     learn.fit(lrs, 1, wds=wd, cycle_len=1, use_clr=(8,3))
#     learn.save(f'fold_{i}_3')

#     learn.freeze_to(-2)
#     learn.fit(lrs, 1, wds=wd, cycle_len=1, use_clr=(8,3))
#     learn.save(f'fold_{i}_3')
    
#     wd = 1e-7
#     learn.unfreeze()
#     learn.fit(lrs, 1, wds=wd, cycle_len=1, use_clr=(8,3))
#     learn.save(f'fold_{i}_3')
    
#     learn.fit(lrs, 1, wds=wd, cycle_len=1)
#     learn.save(f'fold_{i}_3')
    
#     learn.fit(lrs, 1, wds=wd, cycle_len=1)
#     learn.save(f'fold_{i}_3')
    learn.load(f'./10_folds_fwd_3_1.4_86962/fold_{i}_3')
    
    wd = 1e-7
    learn.unfreeze()
    learn.fit(lrs, 1, wds=wd, cycle_len=1, use_clr=(10,10,0.95,0.85))
    learn.save(path+f'fold_{i}_3')
    
    preds = learn.predict(is_test=True)
    
    df = pd.read_csv(CLAS_PATH/'test_.csv', header=None, sep = "\t", quoting=csv.QUOTE_NONE)
    df["prob1"] = [c[1] for c in np.exp(preds)]
    df["prob2"] = [c[2] for c in np.exp(preds)]
    df.to_csv(f"./blend1/_fold_{i}_3.tsv",columns=[2,3,"prob1","prob2"],index=False,sep="\t",header=False)

In [ ]:
#folds neutral
itos = pickle.load((LM_PATH/'tmp'/'itos.pkl').open('rb'))
stoi = collections.defaultdict(lambda:0, {v:k for k,v in enumerate(itos)})

bptt,em_sz,nh,nl = 70,400,1150,3
vs = len(itos)
opt_fn = partial(optim.Adam, betas=(0.8, 0.99))
bs = 48
c = 2
dps = np.array([0.4,0.5,0.05,0.3,0.4])*0.75

lr=1e-3
lrm = 2.6
lrs = np.array([lr/(lrm**4), lr/(lrm**3), lr/(lrm**2), lr/lrm, lr])
#lrs=np.array([1e-4,1e-4,1e-4,1e-3,1e-2])

tst_clas = np.load(CLAS_PATH/'tmp'/'tst_ids_.npy')
tst_labels = np.squeeze(np.load(CLAS_PATH/'tmp'/'tst_labels_.npy'))
tst_ds = TextDataset(tst_clas, tst_labels)
tst_samp = SimpleSampler(tst_clas)
tst_dl = DataLoader(tst_ds, bs, transpose=True, num_workers=1, pad_idx=1, sampler=tst_samp)

for i in tqdm(range(10)):
    trn_clas = np.load(CLAS_PATH/'tmp'/f'trn_ids_{i}_n.npy')
    val_clas = np.load(CLAS_PATH/'tmp'/f'val_ids_{i}_n.npy')
    
    trn_labels = np.squeeze(np.load(CLAS_PATH/'tmp'/f'trn_labels_{i}_n.npy'))
    val_labels = np.squeeze(np.load(CLAS_PATH/'tmp'/f'val_labels_{i}_n.npy'))

    trn_ds = TextDataset(trn_clas, trn_labels)
    val_ds = TextDataset(val_clas, val_labels)
    
    trn_samp = SortishSampler(trn_clas, key=lambda x: len(trn_clas[x]), bs=bs//2)
    val_samp = SortSampler(val_clas, key=lambda x: len(val_clas[x]))
    
    trn_dl = DataLoader(trn_ds, bs//2, transpose=True, num_workers=1, pad_idx=1, sampler=trn_samp)
    val_dl = DataLoader(val_ds, bs, transpose=True, num_workers=1, pad_idx=1, sampler=val_samp)
    
    md = ModelData(PATH, trn_dl, val_dl, tst_dl)

    m = get_rnn_classifer(bptt, 20*70, c, vs, emb_sz=em_sz, n_hid=nh, n_layers=nl, pad_token=1,
              layers=[em_sz*3, 50, c], drops=[dps[4], 0.1],
              dropouti=dps[0], wdrop=dps[1], dropoute=dps[2], dropouth=dps[3])

    opt_fn = partial(optim.Adam, betas=(0.7, 0.99))

    learn = RNN_Learner(md, TextModel(to_gpu(m)), opt_fn=opt_fn)
    learn.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
    learn.clip=25.
    learn.metrics = [accuracy]

#     wd = 0
#     learn.load_encoder('lm_enc1')
#     learn.freeze_to(-1)
#     learn.fit(lrs, 1, wds=wd, cycle_len=1, use_clr=(8,3))
#     learn.save(f'fold_{i}_n')

#     learn.freeze_to(-2)
#     learn.fit(lrs, 1, wds=wd, cycle_len=1, use_clr=(8,3))
#     learn.save(f'fold_{i}_n')
    
#     wd = 1e-7
#     learn.unfreeze()
#     learn.fit(lrs, 1, wds=wd, cycle_len=1, use_clr=(8,3))
#     learn.save(f'fold_{i}_n')
    
#     learn.fit(lrs, 1, wds=wd, cycle_len=1)
#     learn.save(f'fold_{i}_n')
    learn.load(f'./10_folds_fwd_neutral/fold_{i}_n')
    
    wd = 1e-7
    learn.unfreeze()
    learn.fit(lrs, 1, wds=wd, cycle_len=1, use_clr=(10,10,0.95,0.85))
    learn.save(path+f'fold_{i}_n')
    
    preds = learn.predict(is_test=True)
    
    df = pd.read_csv(CLAS_PATH/'test_.csv', header=None, sep = "\t", quoting=csv.QUOTE_NONE)
    df["prob"] = [c[1] for c in np.exp(preds)]
    df.sort_values(by=[2,"prob"], inplace=True, ascending=[True,False])
    df.to_csv(f"./blend1/_fold_{i}_n.tsv",columns=[2,3,"prob"],index=False,sep="\t",header=False)

In [ ]:
#folds reg
itos = pickle.load((LM_PATH/'tmp'/'itos.pkl').open('rb'))
stoi = collections.defaultdict(lambda:0, {v:k for k,v in enumerate(itos)})

bptt,em_sz,nh,nl = 70,400,1150,3
vs = len(itos)
opt_fn = partial(optim.Adam, betas=(0.8, 0.99))
bs = 48
c = 1
dps = np.array([0.4,0.5,0.05,0.3,0.4])*0.75

lr=1e-3
lrm = 2.6
lrs = np.array([lr/(lrm**4), lr/(lrm**3), lr/(lrm**2), lr/lrm, lr])
#lrs=np.array([1e-4,1e-4,1e-4,1e-3,1e-2])

tst_clas = np.load(CLAS_PATH/'tmp'/'tst_ids_.npy')
tst_labels = np.squeeze(np.load(CLAS_PATH/'tmp'/'tst_labels_.npy'))
tst_ds = TextDataset(tst_clas, tst_labels)
tst_samp = SimpleSampler(tst_clas)
tst_dl = DataLoader(tst_ds, bs, transpose=True, num_workers=1, pad_idx=1, sampler=tst_samp)

for i in tqdm(range(10)):
    trn_clas = np.load(CLAS_PATH/'tmp'/f'trn_ids_{i}_r.npy')
    val_clas = np.load(CLAS_PATH/'tmp'/f'val_ids_{i}_r.npy')
    
    trn_labels = np.squeeze(np.load(CLAS_PATH/'tmp'/f'trn_labels_{i}_r.npy'))
    val_labels = np.squeeze(np.load(CLAS_PATH/'tmp'/f'val_labels_{i}_r.npy'))

    trn_ds = TextDataset(trn_clas, trn_labels)
    val_ds = TextDataset(val_clas, val_labels)
    
    trn_samp = SortishSampler(trn_clas, key=lambda x: len(trn_clas[x]), bs=bs//2)
    val_samp = SortSampler(val_clas, key=lambda x: len(val_clas[x]))
    
    trn_dl = DataLoader(trn_ds, bs//2, transpose=True, num_workers=1, pad_idx=1, sampler=trn_samp)
    val_dl = DataLoader(val_ds, bs, transpose=True, num_workers=1, pad_idx=1, sampler=val_samp)
    
    md = ModelData(PATH, trn_dl, val_dl, tst_dl)

    m = get_rnn_regression(bptt, 20*70, c, vs, emb_sz=em_sz, n_hid=nh, n_layers=nl, pad_token=1,
              layers=[em_sz, 50, c], drops=[dps[4], 0.1],
              dropouti=dps[0], wdrop=dps[1], dropoute=dps[2], dropouth=dps[3])

    opt_fn = partial(optim.Adam, betas=(0.7, 0.99))

    learn = RNN_Learner(md, TextModel(to_gpu(m)), opt_fn=opt_fn)
    learn.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
    learn.clip=25.
    
    learn.crit = F.mse_loss

#     wd = 0
#     learn.load_encoder('lm_enc1')
#     learn.freeze_to(-1)
#     learn.fit(lrs, 1, wds=wd, cycle_len=1, use_clr=(8,3))
#     learn.save(f'fold_{i}_r')

#     learn.freeze_to(-2)
#     learn.fit(lrs, 1, wds=wd, cycle_len=1, use_clr=(8,3))
#     learn.save(f'fold_{i}_r')
    
#     wd = 1e-7
#     learn.unfreeze()
#     learn.fit(lrs, 1, wds=wd, cycle_len=1, use_clr=(8,3))
#     learn.save(f'fold_{i}_r')
    
#     learn.fit(lrs, 1, wds=wd, cycle_len=1)
#     learn.save(f'fold_{i}_r')
    
#     learn.fit(lrs, 1, wds=wd, cycle_len=1)
#     learn.save(f'fold_{i}_r')
    
#     learn.fit(lrs, 1, wds=wd, cycle_len=1)
#     learn.save(f'fold_{i}_r')
    learn.load(f'./10_folds_fwd_mse_86488/fold_{i}_r')
    
    wd = 1e-7
    learn.unfreeze()
    learn.fit(lrs, 1, wds=wd, cycle_len=1, use_clr=(10,10,0.95,0.85))
    learn.save(path+f'fold_{i}_r')
    
    preds = learn.predict(is_test=True)
    
    df = pd.read_csv(CLAS_PATH/'test_.csv', header=None, sep = "\t", quoting=csv.QUOTE_NONE)
    df["prob"] = [c[0] for c in preds]
    df.to_csv(f"./blend1/fold_{i}_r.tsv",columns=[2,3,"prob"],index=False,sep="\t",header=False)